In [13]:
# - Importamos librerias necesarias
import requests
import pandas as pd
import datetime
import gspread
import dotenv
import os

In [14]:
# - Cargamos variables de entorno con secretos
try:
    dotenv_path = dotenv.find_dotenv('storage/keys/.env')
    dotenv.load_dotenv(dotenv_path)

except:
    print('No hay variables locales!')

Dotenv localizado y actualizado!


In [15]:
# - Cargamos la hoja de calculo de Google Sheet y consultamos la hoja maestra o de orquestación
gc = gspread.service_account(filename='storage/keys/access-key-google-drive.json')
sheet = gc.open('FIC_dataset')
summary = sheet.worksheet('summary')
df_summary = pd.DataFrame(summary.get_all_records())

In [16]:
if df_summary.loc[0,'value'] == 0:
    print('primera carga')
    print('obtener minDate y maxdate:')
    
    
    # - fecha minima
    query = 'https://www.datos.gov.co/resource/qhpu-8ixx.json?$select=min(fecha_corte)'
    response = requests.get(query)
    
    if response:
        data = response.json()
        minDate = data[0].get('min_fecha_corte')
        print('Request exitoso! - la fecha minima del dataset es: ', minDate)
    else:
        print('Ha ocurrido un error en el Requests de la fecha minima')
    
    # - fecha maxima
    query = 'https://www.datos.gov.co/resource/qhpu-8ixx.json?$select=max(fecha_corte)'
    response = requests.get(query)
    
    if response:
        data = response.json()
        maxDate = data[0].get('max_fecha_corte')
        print('Request exitoso! - la fecha maxima del dataset es: ', maxDate)
    else:
        print('Ha ocurrido un error en el Requests de la fecha maxima')    
        
    # - Convertimos la fecha minima y maxima a formato datetime
    dt_minDate = datetime.datetime.strptime(minDate, '%Y-%m-%dT%H:%M:%S.%f')
    dt_maxDate = datetime.datetime.strptime(maxDate, '%Y-%m-%dT%H:%M:%S.%f')
    
    dt_minDate_inter = dt_minDate
    
    count = 0
    sh_data = sheet.worksheet('data')
    while True:
        
        
        minDate_inter = dt_minDate_inter.strftime('%Y-%m-%dT%H:%M:%S.%f')
        dt_maxDate_inter = dt_minDate_inter + datetime.timedelta(days=90)
        maxDate_inter = dt_maxDate_inter.strftime('%Y-%m-%dT%H:%M:%S.%f')
        
        
        # - Realizamos la primera carga de datos a un df con fecha minima y maxima como parametros
        url = 'https://www.datos.gov.co/resource/qhpu-8ixx.json'
        filtros = f'?$where=fecha_corte between \'{minDate_inter}\' and \'{maxDate_inter}\'&$limit=1500000'
        query = url+filtros
        headers = {'X-App-Token':os.environ.get('X_APP_TOKEN')}
        response = requests.get(query, headers=headers)

        if response:
            data = response.json()
            df = pd.DataFrame(data)
            print('Request exitoso! - DataFrame cargado - dimensiones: ', df.shape)

        else:
            print('Ha ocurrido un error en el Requests de la carga de datos')
        
        # - Enviamos el lote a Google Sheet
        
        if count == 0:
            sh_data.update([df.columns.values.tolist()] + df.values.tolist())
            count = count + 1
        else:
            #sh_data.values_append({'values': df.values.tolist()})
            sheet.values_append('data', {'valueInputOption': 'USER_ENTERED'}, {'values': df.values.tolist()})
            count = count + 1
        
        if dt_maxDate_inter > dt_maxDate:
            break
        else:
            dt_minDate_inter = dt_maxDate_inter + datetime.timedelta(days=1)
        
        print('='*35)
        print('contador:',count)
        print('fecha minima: ',minDate_inter)
        print('fecha maxima: ',maxDate_inter)
        print('tamaño lote df enviado: ',df.shape)
        
            
        
    
    

primera carga
obtener minDate y maxdate:
Request exitoso! - la fecha minima del dataset es:  2016-01-01T00:00:00.000
Request exitoso! - la fecha maxima del dataset es:  2021-03-28T00:00:00.000
Request exitoso! - DataFrame cargado - dimensiones:  (42510, 26)
contador: 1
fecha minima:  2016-01-01T00:00:00.000000
fecha maxima:  2016-03-31T00:00:00.000000
tamaño lote df enviado:  (42510, 26)
Request exitoso! - DataFrame cargado - dimensiones:  (42791, 26)
contador: 2
fecha minima:  2016-04-01T00:00:00.000000
fecha maxima:  2016-06-30T00:00:00.000000
tamaño lote df enviado:  (42791, 26)
Request exitoso! - DataFrame cargado - dimensiones:  (42828, 26)
contador: 3
fecha minima:  2016-07-01T00:00:00.000000
fecha maxima:  2016-09-29T00:00:00.000000
tamaño lote df enviado:  (42828, 26)
Request exitoso! - DataFrame cargado - dimensiones:  (43384, 26)
contador: 4
fecha minima:  2016-09-30T00:00:00.000000
fecha maxima:  2016-12-29T00:00:00.000000
tamaño lote df enviado:  (43384, 26)
Request exitoso

APIError: {'code': 400, 'message': 'This action would increase the number of cells in the workbook above the limit of 5000000 cells.', 'status': 'INVALID_ARGUMENT'}

In [4]:
minDate
#from datetime import datetime

'2016-01-01T00:00:00.000'

In [8]:

print(dt_minDate)
print(dt_maxDate)

2016-01-01 00:00:00
2021-03-28 00:00:00


In [11]:

prev = dt_minDate + datetime.timedelta(days=90)
print(prev)

2016-03-31 00:00:00


In [12]:
if dt_maxDate > prev:
    print('mayor')

mayor


In [ ]:
i = 0
while dt_maxDate > dt_maxDate_requests:
    i = i+1
    print(1)
    

In [34]:
# - Fecha Actual del servidor y conversion a la hora colombiana.
now = datetime.datetime.now(pytz.timezone('America/Bogota'))
fechaEjecucion = now.strftime("%Y-%m-%d %H:%M:%S") 

# - Fechas Interacciones
if fechaInicioInteracciones == '' and fechaFinInteracciones == '':
    # - fecha Inicio
    prev = datetime.datetime.today() + datetime.timedelta(days=-60)
    fechaInicioInteracciones = prev.strftime("%Y-%m-%d")
    # - Fecha Fin 
    fechaFinInteracciones = now.strftime("%Y-%m-%d")
    

21 days, 6:08:00


ERROR: Could not find a version that satisfies the requirement date (from versions: none)
ERROR: No matching distribution found for date


In [22]:
df_summary[0,'value'] == 0:
    #
    #
    #
    

,label,value
0,count,0
1,updateDate,0
2,minDate,0
3,maxDate,0


In [24]:
print(df_summary.loc[0,'value'])

0


In [ ]:
dic_summary = dict(df_summary.values)
dic_summary

In [6]:
# - Realizamos la primera carga de datos a un df con fecha minima y maxima como parametros
minDate = '2016-01-01T00:00:00.000'
maxDate = '2016-01-31T00:00:00.000'

url = 'https://www.datos.gov.co/resource/qhpu-8ixx.json'
filtros = f'?$where=fecha_corte between \'{minDate}\' and \'{maxDate}\'&$limit=1500000'
query = url+filtros
response = requests.get(query)

if response:
    data = response.json()
    df = pd.DataFrame(data)
    print('Request exitoso! - DataFrame cargado - dimensiones: ', df.shape)

else:
    print('Ha ocurrido un error en el Requests de la carga de datos')

Request exitoso! - DataFrame cargado - dimensiones:  (14607, 26)


In [8]:
# - Realizamos la primera carga de datos a un df con fecha minima y maxima como parametros
minDate = '2016-01-01T00:00:00.000'
maxDate = '2016-03-01T00:00:00.000'

url = 'https://www.datos.gov.co/resource/qhpu-8ixx.json'
filtros = f'?$where=fecha_corte between \'{minDate}\' and \'{maxDate}\'&$limit=1500000'
query = url+filtros
response = requests.get(query)

if response:
    data = response.json()
    df = pd.DataFrame(data)
    print('Request exitoso! - DataFrame cargado - dimensiones: ', df.shape)

else:
    print('Ha ocurrido un error en el Requests de la carga de datos')

Request exitoso! - DataFrame cargado - dimensiones:  (28500, 26)


In [9]:
# - Realizamos la primera carga de datos a un df con fecha minima y maxima como parametros
minDate = '2021-01-01T00:00:00.000'
maxDate = '2021-04-01T00:00:00.000'

url = 'https://www.datos.gov.co/resource/qhpu-8ixx.json'
filtros = f'?$where=fecha_corte between \'{minDate}\' and \'{maxDate}\'&$limit=1500000'
query = url+filtros
response = requests.get(query)

if response:
    data = response.json()
    df = pd.DataFrame(data)
    print('Request exitoso! - DataFrame cargado - dimensiones: ', df.shape)

else:
    print('Ha ocurrido un error en el Requests de la carga de datos')

Request exitoso! - DataFrame cargado - dimensiones:  (37558, 26)


In [15]:
# - Realizamos la primera carga de datos a un df con fecha minima y maxima como parametros
minDate = '2021-01-01T00:00:00.000'
maxDate = '2021-04-01T00:00:00.000'

url = 'https://www.datos.gov.co/resource/qhpu-8ixx.json'
filtros = f'?$where=fecha_corte between \'{minDate}\' and \'{maxDate}\'&$limit=1500000'
query = url+filtros
response = requests.get(query)

if response:
    data = response.json()
    df = pd.DataFrame(data)
    print('Request exitoso! - DataFrame cargado - dimensiones: ', df.shape)

else:
    print('Ha ocurrido un error en el Requests de la carga de datos')

Request exitoso! - DataFrame cargado - dimensiones:  (56333, 26)


In [16]:
df.columns

Index(['fecha_corte', 'tipo_entidad', 'nombre_tipo_entidad', 'codigo_entidad',
       'nombre_entidad', 'tipo_negocio', 'nombre_tipo_patrimonio',
       'subtipo_negocio', 'nombre_subtipo_patrimonio', 'codigo_negocio',
       'nombre_patrimonio', 'principal_compartimento', 'tipo_participacion',
       'rendimientos_abonados', 'precierre_fondo_dia_t',
       'numero_unidades_fondo_cierre', 'valor_unidad_operaciones',
       'aportes_recibidos', 'retiros_redenciones', 'anulaciones',
       'valor_fondo_cierre_dia_t', 'numero_inversionistas',
       'rentabilidad_diaria', 'rentabilidad_mensual', 'rentabilidad_semestral',
       'rentabilidad_anual'],
      dtype='object')

In [17]:
# convert the 'Date' column to datetime format
df['fecha_corte']= pd.to_datetime(df['fecha_corte'])

In [21]:
df.groupby('fecha_corte').count()

,tipo_entidad,nombre_tipo_entidad,codigo_entidad,nombre_entidad,tipo_negocio,nombre_tipo_patrimonio,subtipo_negocio,nombre_subtipo_patrimonio,codigo_negocio,nombre_patrimonio,...,valor_unidad_operaciones,aportes_recibidos,retiros_redenciones,anulaciones,valor_fondo_cierre_dia_t,numero_inversionistas,rentabilidad_diaria,rentabilidad_mensual,rentabilidad_semestral,rentabilidad_anual
fecha_corte,,,,,,,,,,,,,,,,,,,,,
2021-01-01,177,177,177,177,177,177,177,177,177,177,...,177,177,177,177,177,177,177,177,177,177
2021-01-02,177,177,177,177,177,177,177,177,177,177,...,177,177,177,177,177,177,177,177,177,177
2021-01-03,177,177,177,177,177,177,177,177,177,177,...,177,177,177,177,177,177,177,177,177,177
2021-01-04,177,177,177,177,177,177,177,177,177,177,...,177,177,177,177,177,177,177,177,177,177
2021-01-05,692,692,692,692,692,692,692,692,692,692,...,692,692,692,692,692,692,692,692,692,692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-24,671,671,671,671,671,671,671,671,671,671,...,671,671,671,671,671,671,671,671,671,671
2021-03-25,699,699,699,699,699,699,699,699,699,699,...,699,699,699,699,699,699,699,699,699,699
2021-03-26,700,700,700,700,700,700,700,700,700,700,...,700,700,700,700,700,700,700,700,700,700


In [23]:


df['fecha_corte'].dt.quarter

0        1
1        1
2        1
3        1
4        1
        ..
56328    1
56329    1
56330    1
56331    1
56332    1
Name: fecha_corte, Length: 56333, dtype: int64

In [26]:
df.groupby(df['fecha_corte'].dt.month).count()

,fecha_corte,tipo_entidad,nombre_tipo_entidad,codigo_entidad,nombre_entidad,tipo_negocio,nombre_tipo_patrimonio,subtipo_negocio,nombre_subtipo_patrimonio,codigo_negocio,...,valor_unidad_operaciones,aportes_recibidos,retiros_redenciones,anulaciones,valor_fondo_cierre_dia_t,numero_inversionistas,rentabilidad_diaria,rentabilidad_mensual,rentabilidad_semestral,rentabilidad_anual
fecha_corte,,,,,,,,,,,,,,,,,,,,,
1,18315,18315,18315,18315,18315,18315,18315,18315,18315,18315,...,18315,18315,18315,18315,18315,18315,18315,18315,18315,18315
2,18543,18543,18543,18543,18543,18543,18543,18543,18543,18543,...,18543,18543,18543,18543,18543,18543,18543,18543,18543,18543
3,19475,19475,19475,19475,19475,19475,19475,19475,19475,19475,...,19475,19475,19475,19475,19475,19475,19475,19475,19475,19475


In [14]:
# importing pandas as pd
import pandas as pd
 
# Creating the dataframe
df = pd.DataFrame({'Date':['11/8/2011', '04/23/2008', '10/2/2019'],
                'Event':['Music', 'Poetry', 'Theatre'],
                'Cost':[10000, 5000, 15000]})
 
 
# Now we will check the data type
# of the 'Date' column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    3 non-null      object
 1   Event   3 non-null      object
 2   Cost    3 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 200.0+ bytes


In [4]:
# - Realizamos la primera carga de datos a un df con fecha minima y maxima como parametros
url = 'https://www.datos.gov.co/resource/qhpu-8ixx.json?$limit=50000'
#filtros = f'?$where=fecha_corte between \'{minDate}\' and \'{maxDate}\'&$limit=1500000'
#query = url+filtros
#headers = {'X-App-Token':os.environ.get('X_APP_TOKEN')}
response = requests.get(url)

if response:
    data = response.json()
    df = pd.DataFrame(data)
    print('Request exitoso! - DataFrame cargado - dimensiones: ', df.shape)

else:
    print('Ha ocurrido un error en el Requests de la carga de datos')

Request exitoso! - DataFrame cargado - dimensiones:  (50000, 26)


In [5]:
# - Cargamos la hoja de calculo de Google Sheet y consultamos sus hojas
gc = gspread.service_account(filename='storage/keys/access-key-google-drive.json')
sheet = gc.open('FIC_dataset')
print(sheet.worksheets())

# - Cargamos la hoja data y enviamos el df a la hoja data con la primera carga
sh_data = sheet.worksheet('data')
sh_data.update([df.columns.values.tolist()] + df.values.tolist())

[<Worksheet 'summary' id:0>, <Worksheet 'data' id:1555986092>]


{'spreadsheetId': '1flfqQEOwde0c0jCgLnfSk4te_oWN-com-7LPvVe4NOo',
 'updatedRange': 'data!A1:Z50001',
 'updatedRows': 50001,
 'updatedColumns': 26,
 'updatedCells': 1300026}

In [ ]:
import pandas as pd

for i in range(10):
    locals()['df' + str(i)] = pd.DataFrame()
    print(locals()['df' + str(i)].shape)